In [14]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, LSTM, Dense, Concatenate, Dropout
from keras.models import Model
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the dataset
df = pd.read_csv('/content/Laptop_Train_v2 (2).csv')
word_to_drop = 'conflict'
column_name = 'polarity'
df = df[~df['polarity'].str.contains('conflict')]
df = df.drop(['id','from','to'], axis = 1)
map_polarity = {'neutral': 2, 'positive': 1, "negative": 0} # map data

df['polarity']= df['polarity'].map(map_polarity)



In [15]:
# import nltk
# import re
# from nltk.corpus import stopwords
# nltk.download('stopwords')
# from nltk.tokenize import word_tokenize
# import nltk
# nltk.download('punkt')
# from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer
# ps = PorterStemmer()

In [16]:
# def preprocess_text(Sentence):
#     # Remove special characters and digits
#     Sentence = re.sub('[^a-zA-Z]', ' ', Sentence)
#     # Convert to lowercase
#     Sentence = Sentence.lower()
#     # Tokenize the text
#     tokens = word_tokenize(Sentence)
#     # Remove stopwords
#     tokens = [word for word in tokens if word not in stopwords.words('english')]
#     # Join the tokens back into a string
#     Sentence = ' '.join(tokens)
#     return Sentence

In [17]:
# df['Sentence']= df['Sentence'].apply(preprocess_text)

In [18]:
# Aspect=df['Aspect Term'].apply(preprocess_text)
# df['Aspect Term']=df['Aspect Term'].lower()

In [19]:
X = df['Sentence'].values
y = df['polarity'].values
aspects = df['Aspect Term'].values

# Tokenize the reviews and aspects
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(np.concatenate([X, aspects]))
X_seq = tokenizer.texts_to_sequences(X)
aspects_seq = tokenizer.texts_to_sequences(aspects)

# Pad the sequences
max_len = 50
X_pad = pad_sequences(X_seq, maxlen=max_len, padding='post', truncating='post')
aspects_pad = pad_sequences(aspects_seq, maxlen=max_len, padding='post', truncating='post')




In [20]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test, aspects_train, aspects_test = train_test_split(X_pad, y, aspects_pad, test_size=0.2, random_state=42)

# Convert the labels to one-hot encoding
y_train_onehot = to_categorical(y_train)
y_test_onehot = to_categorical(y_test)



In [21]:
# Define the LSTM model
inputs1 = Input(shape=(max_len,))
embedding1 = Embedding(input_dim=10000, output_dim=128, input_length=max_len)(inputs1)
lstm1 = LSTM(64, dropout=0.2, recurrent_dropout=0.2)(embedding1)

inputs2 = Input(shape=(max_len,))
embedding2 = Embedding(input_dim=10000, output_dim=128, input_length=max_len)(inputs2)
lstm2 = LSTM(64, dropout=0.2, recurrent_dropout=0.2)(embedding2)

concat = Concatenate()([lstm1, lstm2])
dense = Dense(64, activation='relu')(concat)
dropout = Dropout(0.2)(dense)
output = Dense(3, activation='softmax')(dropout)

In [22]:
model = Model(inputs=[inputs1, inputs2], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([X_train, aspects_train], y_train_onehot, validation_split=0.2, epochs=20, batch_size=32)


Epoch 1/20
47/47 [==============================] - 22s 290ms/step - loss: 1.0508 - accuracy: 0.4277 - val_loss: 1.0437 - val_accuracy: 0.4162
Epoch 2/20
47/47 [==============================] - 12s 262ms/step - loss: 1.0377 - accuracy: 0.4595 - val_loss: 0.9366 - val_accuracy: 0.5973
Epoch 3/20
47/47 [==============================] - 11s 243ms/step - loss: 0.8650 - accuracy: 0.6372 - val_loss: 0.9512 - val_accuracy: 0.6027
Epoch 4/20
47/47 [==============================] - 10s 210ms/step - loss: 0.8204 - accuracy: 0.6493 - val_loss: 1.0688 - val_accuracy: 0.4892
Epoch 5/20
47/47 [==============================] - 11s 225ms/step - loss: 0.8215 - accuracy: 0.6574 - val_loss: 0.9352 - val_accuracy: 0.6243
Epoch 6/20
47/47 [==============================] - 11s 244ms/step - loss: 0.7190 - accuracy: 0.7243 - val_loss: 1.0102 - val_accuracy: 0.6054
Epoch 7/20
47/47 [==============================] - 11s 232ms/step - loss: 0.6657 - accuracy: 0.7419 - val_loss: 0.9879 - val_accuracy: 0.6135

In [23]:
# Test the model
y_pred_onehot = model.predict([X_test, aspects_test])
y_pred = np.argmax(y_pred_onehot, axis=1)
accuracy = accuracy_score(y_test, y_pred)


15/15 [==============================] - 2s 44ms/step


In [12]:
f1 = f1_score(y_test, y_pred)

print(f"Test Accuracy: {accuracy}")
print(f"Test F1 Score: {f1}")

ValueError: ignored